# Baseline

In [5]:
import time
import numpy as np
import pandas as pd

## df 节省内存函数

In [6]:
def reduce_mem(df):
    start_time = time.time()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    # 返回的值以字节为单位，转换成以 MB 为单位
    start_mem = df.memery_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_max, c_min = df[col].max(), df[col].min()
            if pd.isnull(c_min) or pd.isnull(c_max):
                continue
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memery_usage().sum() / 1024**2
    print(f'现在数据占用大小为: {end_mem}')
    print(f'减少了{100 * (start_mem - end_mem) / start_mem}')
    print(f'花费时间: {(time.time() - start_time) / 60}')
    return df

## 读取采样或全量数据

In [18]:
def get_all_click_sample(data_path: str, sample_nums: int=10000) -> pd.DataFrame:
    """
    训练集中采样一部分数据调试。
    
    Args:
        data_path(`str`): 原数据的存储路径
        sample_nums(`int`): 采样数目（这里由于机器的内存限制，可以采样用户做）

    Returns:
        pd.DataFrame: 返回一个包含采样后的点击日志数据的 DataFrame；
                      该数据只包含随机选择的用户的点击记录，并且去除了重复的点击记录。
    """
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    all_user_ids = all_click.user_id.unique()

    sample_user_ids = np.random.choice(all_user_ids, size=sample_nums, replace=False) 
    all_click = all_click[all_click['user_id'].isin(sample_user_ids)]
    
    all_click = all_click.drop_duplicates((['user_id', 'click_article_id', 'click_timestamp']))
    return all_click


def get_all_click_df(data_path: str='../data/', offline: bool=True) -> pd.DataFrame:
    """
    读取点击数据，支持线上和线下数据的获取。
    根据  `offline` 参数的值决定读取训练集或训练集与测试集的合并数据。

    Args:
        data_path (`str`): 数据存储路径，默认为 './data_raw/'。
        offline (`bool`): 是否仅使用训练集数据。若为 True，则只读取训练集；若为 False，则合并训练集和测试集。

    Returns:
        pd.DataFrame: 返回一个包含用户点击日志的 DataFrame，数据中去除了重复的点击记录。
    """
    all_click = pd.read_csv(data_path + 'train_click_log.csv')
    if not offline:
        tst_click = pd.read_csv(data_path + 'testA_click_log.csv')
        all_click = pd.concat([all_click, tst_click], ignore_index=True)
    
    all_click = all_click.drop_duplicates(['user_id', 'click_article_id', 'click_timestamp'])
    return all_click

In [21]:
# 获取全量训练集
all_click_df = get_all_click_df(offline=False)
all_click_df

,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
3,199998,157770,1507029532200,4,1,17,1,25,5
4,199998,96613,1507029671831,4,1,17,1,25,5
...,...,...,...,...,...,...,...,...,...
1630628,221924,70758,1508211323220,4,3,2,1,25,2
1630629,207823,331116,1508211542618,4,3,2,1,25,1
1630630,207823,234481,1508211850103,4,3,2,1,25,1
1630631,207823,211442,1508212189949,4,3,2,1,25,1


## 获取【用户-文章-点击时间】字典

In [22]:
def get_user_item_time(click_df: pd.DataFrame) -> dict:
    """
    根据点击时间获取用户的点击文章序列。
    函数将每个用户的点击文章和对应的点击时间组合成字典，格式为 
    {user1: {item1: time1, item2: time2, ...}, ...}。

    Args:
        click_df (pd.DataFrame): 包含用户点击数据的 DataFrame

    Returns:
        dict: 返回一个字典，其中键是用户 ID，值是另一个字典，包含用户点击的文章及其对应的点击时间。
    """
    click_df = click_df.sort_values('click_timestamp')
    
    user_item_time_df = click_df.groupby('user_id')['click_article_id', 'click_timestamp'].apply(
        lambda x: list(zip(x['click_article_id'], x['click_timestamp']))
    ).reset_index().rename(columns={0: 'item_time_list'})
    
    # 将用户 ID 和对应的点击文章时间列表转换为字典
    user_item_time_dict = dict(zip(user_item_time_df['user_id'], user_item_time_df['item_time_list']))
    
    return user_item_time_dict

## 获取点击最多的 TopK 个文章

In [23]:
def get_item_topk_click(click_df: pd.DataFrame, k: int) -> pd.Series:
    """
    获取点击次数最多的前 k 个文章 ID。

    Args:
        click_df (`pd.DataFrame`): 包含点击数据的 DataFrame，必须包含 'click_article_id' 列。
        k (`int`): 需要返回的前 k 个文章的数量。

    Returns:
        `pd.Series`: 返回一个包含前 k 个点击次数最多的文章 ID 的 Series。
    """
    return click_df['click_article_id'].value_counts().index[:k]